# Coastal Flood Risk Visualization

This visualization uses a gpu to plot households and tanks on a dashboard, with range slider features to show households that have a certain risk index range. Also we have included the elderly multiselect, and household/tank distance slider.

### Import Statements

In [1]:
import geopandas as gpd

import cuxfilter
from cuxfilter.layouts import feature_and_five_edge, double_feature
import cudf
import numpy as np

import holoviews as hv
import pandas as pd

from pyproj import Proj, Transformer



### Importing Hurricane Risk and Distance Data

This is a preprocessed parquet file that includes the latitude and longitude of households and tanks that are already transformed into the 3857 projections. This file also contains the ```is_elderly``` variable that indicates if the point at a particular lat/lon is a household with elderly, a household without elderly, or a tank. ```distance_mi``` is the distance in miles between tanks and houses. 

In [5]:
df = pd.read_parquet('/hpc/group/codeplus22-vis/infousa_copy/coast_flood_risks_final.parquet')
df

,is_elderly,distance_mi,coastal_flood_risk,lat_3857,lon_3857
0,2,33.459368,-1.0,-8.509454e+06,5.101307e+06
1,2,28.501947,-1.0,-8.499018e+06,5.096218e+06
2,2,28.592896,-1.0,-8.496356e+06,5.099448e+06
3,2,33.876175,-1.0,-8.509963e+06,5.103102e+06
4,2,33.117674,-1.0,-8.508370e+06,5.099066e+06
...,...,...,...,...,...
53165525,0,215.000000,-1.0,-1.167972e+07,4.833652e+06
53165526,0,215.000000,-1.0,-1.167971e+07,4.833633e+06
53165527,0,215.000000,-1.0,-1.167971e+07,4.833691e+06
53165528,0,215.000000,-1.0,-1.167971e+07,4.833535e+06


### Transforming the dataframe type

Here, we are transforming the pandas dataframe into a cudf dataframe and then into a cuxfilter dataframe so that we can plot the points using the cuxfilter library.

In [6]:
cdf = cudf.DataFrame.from_pandas(df) 

In [7]:
cux_df = cuxfilter.DataFrame.from_dataframe(cdf) 

### Creating multiselect label map and defining map color palette

In the label map for the ```is_elderly``` multiselect, we are defining the labels for the values that equal 0, 1, and 2 respectively.

In [10]:

label_elderly = {0: 'Tank', 1: 'Elderly', 
             2: 'No Elderly'}

from bokeh.palettes import Spectral11
colors = list(Spectral11)
colors.reverse()


### Defining charts

Below, we are defining the four charts that will be plotted on our dashboard. In each chart, you can define the type of cuxfilter chart (```scatter```, ```multiselect```, ```range_slider```, ```drop_down```, etc.) and also specify the column that the chart is made on. The label map parameter specifies which label map the feature will pull from. 

In [16]:
household = cuxfilter.charts.scatter(x='lat_3857', y='lon_3857', pixel_shade_type='linear', color_palette = colors, aggregate_fn = 'max', aggregate_col = 'distance_mi', tile_provider="CartoDark", title = 'Households and distances to nearest tank, with coastal flooding risk slider', legend = True)

elderly = cuxfilter.charts.multi_select('is_elderly', label_map = label_elderly)

coast_flood_risk= cuxfilter.charts.range_slider('coastal_flood_risk')

dist = cuxfilter.charts.range_slider('distance_mi')


### Plotting the charts on the dashboard

The layout and sidebar parameter determines how the charts are oganized on the dashboard. Here we have put three charts as the sidebar, however, you could also place those three charts underneath the main chart by specifying feature and base under the layout feature.

In [12]:
d = cux_df.dashboard([household], sidebar = [elderly, coast_flood_risk, dist], layout = cuxfilter.layouts.feature_and_base, theme = cuxfilter.themes.rapids) 


### Display the visualization

In [17]:
d.show()
d.app(sidebar_width = 250)

Dashboard running at port 51641


Traceback (most recent call last): 
 File "/opt/conda/envs/rapids/lib/python3.9/site-packages/holoviews/plotting/util.py", line 277, in get_plot_frame 
 return map_obj[key] 
 File "/opt/conda/envs/rapids/lib/python3.9/site-packages/holoviews/core/spaces.py", line 1341, in __getitem__ 
 val = self._execute_callback(*tuple_key) 
 File "/opt/conda/envs/rapids/lib/python3.9/site-packages/holoviews/core/spaces.py", line 1110, in _execute_callback 
 retval = self.callback(*args, **kwargs) 
 File "/opt/conda/envs/rapids/lib/python3.9/site-packages/holoviews/core/spaces.py", line 683, in __call__ 
 if not args and not kwargs and not any(kwarg_hash): return self.callable() 
 File "/opt/conda/envs/rapids/lib/python3.9/site-packages/holoviews/util/__init__.py", line 1043, in dynamic_operation 
 key, obj = resolve(key, kwargs) 
 File "/opt/conda/envs/rapids/lib/python3.9/site-packages/holoviews/util/__init__.py", line 1032, in resolve 
 return key, map_obj[key] 
 File "/opt/conda/envs/rapids/lib/python3.9/site-packages/holoviews/core/spaces.py", line 1341, in __getitem__ 
 val = self._execute_callback(*tuple_key) 
 File "/opt/conda/envs/rapids/lib/python3.9/site-packages/holoviews/core/spaces.py", line 1110, in _execute_callback 
 retval = self.callback(*args, **kwargs) 
 File "/opt/conda/envs/rapids/lib/python3.9/site-packages/holoviews/core/spaces.py", line 683, in __call__ 
 if not args and not kwargs and not any(kwarg_hash): return self.callable() 
 File "/opt/conda/envs/rapids/lib/python3.9/site-packages/holoviews/core/overlay.py", line 31, in dynamic_mul 
 element = other[args] 
 File "/opt/conda/envs/rapids/lib/python3.9/site-packages/holoviews/core/spaces.py", line 1341, in __getitem__ 
 val = self._execute_callback(*tuple_key) 
 File "/opt/conda/envs/rapids/lib/python3.9/site-packages/holoviews/core/spaces.py", line 1110, in _execute_callback 
 retval = self.callback(*args, **kwargs) 
 File "/opt/conda/envs/rapids/lib/python3.9/site-packages/holoviews/core/spaces.py", line 683, in __call__ 
 if not args and not kwargs and not any(kwarg_hash): return self.callable() 
 File "/opt/conda/envs/rapids/lib/python3.9/site-packages/holoviews/core/spaces.py", line 210, in dynamic_mul 
 self_el = self.select(HoloMap, **key_map) if self.kdims else self[()] 
 File "/opt/conda/envs/rapids/lib/python3.9/site-packages/holoviews/core/spaces.py", line 1341, in __getitem__ 
 val = self._execute_callback(*tuple_key) 
 File "/opt/conda/envs/rapids/lib/python3.9/site-packages/holoviews/core/spaces.py", line 1110, in _execute_callback 
 retval = self.callback(*args, **kwargs) 
 File "/opt/conda/envs/rapids/lib/python3.9/site-packages/holoviews/core/spaces.py", line 683, in __call__ 
 if not args and not kwargs and not any(kwarg_hash): return self.callable() 
 File "/opt/conda/envs/rapids/lib/python3.9/site-packages/holoviews/util/__init__.py", line 1043, in dynamic_operation 
 key, obj = resolve(key, kwargs) 
 File "/opt/conda/envs/rapids/lib/python3.9/site-packages/holoviews/util/__init__.py", line 1032, in resolve 
 return key, map_obj[key] 
 File "/opt/conda/envs/rapids/lib/python3.9/site-packages/holoviews/core/spaces.py", line 1341, in __getitem__ 
 val = self._execute_callback(*tuple_key) 
 File "/opt/conda/envs/rapids/lib/python3.9/site-packages/holoviews/core/spaces.py", line 1110, in _execute_callback 
 retval = self.callback(*args, **kwargs) 
 File "/opt/conda/envs/rapids/lib/python3.9/site-packages/holoviews/core/spaces.py", line 683, in __call__ 
 if not args and not kwargs and not any(kwarg_hash): return self.callable() 
 File "/opt/conda/envs/rapids/lib/python3.9/site-packages/holoviews/util/__init__.py", line 1043, in dynamic_operation 
 key, obj = resolve(key, kwargs) 
 File "/opt/conda/envs/rapids/lib/python3.9/site-packages/holoviews/util/__init__.py", line 1032, in resolve 
 return key, map_obj[key] 
 File "/opt/conda/envs/rapids/lib/python3.9/site-packages/holoviews/core/spaces.py", line 1341, in __getitem__ 
 val = self._execute_ca

Row(sizing_mode='stretch_both')
    [0] Column(sizing_mode='stretch_height', width=250)
        [0] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='Datapoints Selected')
            [0] Column(sizing_mode='stretch_both')
                [0] Number(css_classes=['indicator'], font_size='18pt', format='{value:,}', sizing_mode='stretch_width', value=53165530)
                [1] Progress(sizing_mode='stretch_width', value=100)
        [1] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='is_elderly_multi_select', width=400)
            [0] MultiSelect(height=200, options={'Tank': 0, 'Elderly': 1, ...}, sizing_mode='stretch_both', value=[''], width=400)
        [2] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='coastal_flood_risk_range_..., width=400)
            [0] RangeSlider(end=94.00797588572877, sizing_mode='stretch_both', start=-1.0, value=(-1.0, 94.00797588572877), value_end=94.00797588572877, value_start=-1.0)
        [3] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='distance_mi_range_slider'..., width=400)
            [0] RangeSlider(end=215.0, sizing_mode='stretch_both', start=0.000592927239086057, value=(0.000592927239086057, ..., value_end=215.0, value_start=0.000592927239086057)
    [1] GridStack(allow_drag=False, allow_resize=False, ncols=11, nrows=4, sizing_mode='stretch_both')
        [0] Card(sizing_mode='stretch_both', title='Households and distances ..., width=800)
            [0] HoloViews(DynamicMap, height=400, sizing_mode='stretch_both')